In [33]:
import math
import pandas as pd

In [34]:
df = pd.read_csv("../data/babyNamesUSYOB-full.csv")

In [35]:
grouped_by_sex = df.groupby(["YearOfBirth", "Sex"])["Number"].sum().reset_index()

In [36]:
def add_both(x):
    last_row = x.iloc[-1].copy()
    last_row["Sex"] = "Both"
    last_row["Number"] = int(x[x["Sex"] == "M"]["Number"]) + int(x[x["Sex"] == "F"]["Number"])
    return x.append(last_row)
total_births = grouped_by_sex.groupby("YearOfBirth").apply(add_both).reset_index(drop=True)

C:\Users\Rahim\AppData\Local\Temp\ipykernel_9992\2081189117.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return x.append(last_row)
C:\Users\Rahim\AppData\Local\Temp\ipykernel_9992\2081189117.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return x.append(last_row)
C:\Users\Rahim\AppData\Local\Temp\ipykernel_9992\2081189117.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return x.append(last_row)
C:\Users\Rahim\AppData\Local\Temp\ipykernel_9992\2081189117.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return x.append(last_row)
C:\Users\Rahim\AppData\Local\Temp\ipykernel_9992\2081189117.py:5: FutureWarning: The frame.a

In [37]:
total_births.to_csv("../data/total_births.csv", index=False)

In [38]:
unique_names = df.groupby(["YearOfBirth", "Sex"])["Name"].count().reset_index()

In [39]:
unique_names_both = df.drop(["Sex", "Number"], axis=1).drop_duplicates().groupby("YearOfBirth")["Name"].count().reset_index()

In [40]:
def add_both_unique(x):
    last_row = x.iloc[-1].copy()
    last_row["Sex"] = "Both"
    last_row["Name"] = int(unique_names_both[unique_names_both["YearOfBirth"] == last_row["YearOfBirth"]]["Name"])
    return x.append(last_row)
unique_names = unique_names.groupby("YearOfBirth").apply(add_both_unique).reset_index(drop=True)

C:\Users\Rahim\AppData\Local\Temp\ipykernel_9992\124520823.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return x.append(last_row)
C:\Users\Rahim\AppData\Local\Temp\ipykernel_9992\124520823.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return x.append(last_row)
C:\Users\Rahim\AppData\Local\Temp\ipykernel_9992\124520823.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return x.append(last_row)
C:\Users\Rahim\AppData\Local\Temp\ipykernel_9992\124520823.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return x.append(last_row)
C:\Users\Rahim\AppData\Local\Temp\ipykernel_9992\124520823.py:5: FutureWarning: The frame.append

In [41]:
unique_names = unique_names.rename(columns={"Name":"Count"})

In [42]:
unique_names.to_csv("../data/unique_names.csv", index=False)

In [43]:
def add_prop(group):
    group["Proportion"] = group.Number / group.Number.sum()
    return group

name_records = df.groupby(["YearOfBirth","Sex"]).apply(add_prop)

C:\Users\Rahim\AppData\Local\Temp\ipykernel_9992\911188375.py:5: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  name_records = df.groupby(["YearOfBirth","Sex"]).apply(add_prop)


In [44]:
name_records["NameLength"] = name_records["Name"].apply(lambda x: len(x))

In [45]:
name_records["Decade"] = name_records["YearOfBirth"].apply(lambda x: math.floor(x/10)*10)
name_records.head()

,YearOfBirth,Name,Sex,Number,Proportion,NameLength,Decade
0,1880,Mary,F,7065,0.077644,4,1880
1,1880,Anna,F,2604,0.028618,4,1880
2,1880,Emma,F,2003,0.022013,4,1880
3,1880,Elizabeth,F,1939,0.021310,9,1880
4,1880,Minnie,F,1746,0.019189,6,1880


In [46]:
name_records.to_csv("../data/name_records.csv", index=False)

In [52]:
def eliminate_non_unisex_names(group):
    if group.shape[0] == 1:
        return None
    else:
        name = group["Name"].unique()[0]
        group = pd.DataFrame(data={"Name": [name], "Number": [group["Number"].min()]})
        return group
unisex_names = name_records.groupby(["Name", "Sex"])["Number"].sum().reset_index().groupby("Name").apply(eliminate_non_unisex_names).reset_index(drop=True).sort_values(by=["Number"], ascending=False).reset_index(drop=True)

In [54]:
unisex_names.to_csv("../data/unisex_names.csv", index=False)